<a href="https://colab.research.google.com/github/daniilprigozhin/ProteinFamily/blob/main/Protein_Family_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Top of the file

In [1]:
!ls

sample_data
